In [ ]:
import os 
import sys
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# YouTube API Configuration
youtube_api_key = os.getenv('YOUTUBE_API_KEY')
channel_id = os.getenv('YOUTUBE_CHANNEL_ID')



def validate_config():
    """Validate all required configuration"""
    errors = []
    
    # Check YouTube config
    if not youtube_api_key:
        errors.append("Missing YouTube API key")
    if not channel_id:
        errors.append("Missing YouTube Channel ID")
        
    # Check Database config
    required_db_keys = ['host', 'database', 'user', 'password']
    for key in required_db_keys:
        if not db_config.get(key):
            errors.append(f"Missing database {key}")
    
    if errors:
        print("Configuration errors:", file=sys.stderr)
        for error in errors:
            print(f"× {error}", file=sys.stderr)
        sys.exit(1)

validate_config()

# Initialize YouTube API client
youtube = build('youtube', 'v3', developerKey=youtube_api_key)

def get_uploads_playlist(channel_id):
    """Get the uploads playlist ID for a channel"""
    try:
        response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()
        
        if 'items' in response and len(response['items']) > 0:
            return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        return None
    except Exception as e:
        print(f'× Error fetching playlist ID: {e}', file=sys.stderr)
        return None

def get_video_ids_from_playlist(playlist_id, max_results=50):
    """Get all video IDs from a playlist"""
    video_ids = []
    try:
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=max_results
        )
        
        while request:
            response = request.execute()
            video_ids.extend([item['contentDetails']['videoId'] for item in response.get('items', [])])
            
            request = youtube.playlistItems().list_next(request, response)
            
        return video_ids
    except Exception as e:
        print(f'× Error fetching video IDs: {e}', file=sys.stderr)
        return None

def get_video_details(video_ids):
    """Get detailed information for a list of video IDs"""
    video_data = []
    
    # Process videos in batches of 50 (YouTube API limit)
    for i in range(0, len(video_ids), 50):
        try:
            # Join the next batch of 50 video IDs
            id_batch = ','.join(video_ids[i:i+50])
            
            # Make API request for video details
            response = youtube.videos().list(
                part='snippet,statistics,contentDetails',
                id=id_batch
            ).execute()
            
            # Process each video in the response
            for item in response.get('items', []):
                snippet = item.get('snippet', {})
                stats = item.get('statistics', {})
                content_details = item.get('contentDetails', {})
                
                video_data.append({
                    'video_id': item['id'],
                    'title': snippet.get('title', ''),
                    'published_at': snippet.get('publishedAt', ''),
                    'view_count': int(stats.get('viewCount', 0)),
                    'like_count': int(stats.get('likeCount', 0)),
                    'comment_count': int(stats.get('commentCount', 0)),
                    'description': snippet.get('description', ''),
                    'thumbnail_url': snippet.get('thumbnails', {}).get('high', {}).get('url', ''),
                    'duration': content_details.get('duration', ''),
                    'channel_id': snippet.get('channelId', ''),
                    'channel_title': snippet.get('channelTitle', '')
                })
                
        except Exception as e:
            print(f"Error processing videos {i} to {i+50}: {str(e)}", file=sys.stderr)
            continue
    
    return video_data

if __name__ == "__main__":
    # Get the uploads playlist ID
    playlist_id = get_uploads_playlist(channel_id)
    if not playlist_id:
        print("Failed to get uploads playlist ID")
        sys.exit('No playlist found for the given channel_ID')
    
    print(f"Uploads playlist ID: {playlist_id}")
    
    # Get all video IDs from the playlist
    video_ids = get_video_ids_from_playlist(playlist_id)
    if not video_ids:
        print("Failed to get video IDs from playlist")
        sys.exit('No videos IDS found')
    
    print(f"Found {len(video_ids)} videos in playlist")
    
    # Get detailed information for all videos
    video_details = get_video_details(video_ids)
    if video_details:
        print(f"\nSuccessfully retrieved details for {len(video_details)} videos")
        print("\nSample video data:")
        print(json.dumps(video_details[0], indent=2))
    else:
        print("Failed to get video details")
    # Save to JSON file
        output_file = 'youtube_videos.json'  # Simplified path - will save in current directory
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(video_details, f, indent=4, ensure_ascii=False)
            print(f"\nData successfully saved to {output_file}")
        except Exception as e:
            print(f"\n× Error saving data to file: {e}", file=sys.stderr)
        else:
            print("Failed to get video details")

ModuleNotFoundError: No module named 'googleapiclient'